In [1]:
import sys
print(sys.getrecursionlimit())

3000


In [2]:
from rich.console import Console
# Create a safe console for Jupyter (no ANSI, no recursion)
console = Console(force_terminal=False, file=sys.__stdout__)

In [3]:
import resource, sys
resource.setrlimit(resource.RLIMIT_STACK, (2**29,-1))
sys.setrecursionlimit(10**6)

In [4]:
import os
os.environ["NO_COLOR"]="1"   # Disable color output for rich library

In [9]:
# insurer_agents.py
from crewai import Agent, Task, Crew, LLM, Process
from tools import OCRTool, JSONValidatorTool, FHIRParserTool, CostEstimatorTool, CreateUUIDTool
from langchain.chat_models import ChatOpenAI
from langchain.agents import Tool
from textwrap import dedent

# Shared LLM for all agents
llm = LLM(model="ollama/deepseek-r1:1.5b", base_url="http://localhost:11434")

# Instantiate Tool Definitions
ocr_tool_instance = OCRTool()
json_validator_tool_instance = JSONValidatorTool()
fhir_parser_tool_instance = FHIRParserTool()
cost_estimator_tool_instance = CostEstimatorTool()


# 1. PreAuth_Intake_Agent
intake_agent = Agent(
    role="Intelligent Intake Processor for Preauthorization Requests",
    goal=dedent("""
        Efficiently ingest and standardize preauthorization request data across multiple formats,
        ensuring all essential fields are extracted, validated, and structured for downstream decision-making.
    """),
    backstory=dedent("""
        You are an advanced intake specialist built to handle diverse input formats such as JSON, FHIR,
        HL7, PDFs, or scanned documents. Your responsibilities include:
        - Parsing and extracting key fields like patient name, age, diagnosis, ICD and CPT codes,
          insurer, member id, policy number, and estimated cost.
        - If given an image path, use OCR tools to interpret scanned forms for structured clinical data.
        - If given a JSON, parse the JSON and validate completeness and consistency of all required fields using JSON schema validation tools.
        - Assigning a unique case ID and logging metadata for audit traceability.
        Your performance is critical to ensuring accuracy and completeness before the data reaches
        policy, clinical, and cost review stages.
    """),
    # tools=[json_validator_tool_instance],
    input_keys = ["preauth_request"],
    llm=llm
)

# 2. Policy_Validation_Agent
policy_agent = Agent(
    role="Eligibility and Coverage Validation Agent",
    goal=dedent("""
        Accurately determine if the patient's insurance policy covers the requested procedure by verifying
        member identity, policy status, and plan-specific rules, including exclusions, co-pays, and limits.
    """),
    backstory=dedent("""
        You specialize in interpreting and validating insurance policy details to ensure that only eligible,
        in-policy requests advance through the preauthorization process. Your responsibilities include:
        - Matching member ID and policy number against insurer records.
        - Verifying that the policy is active and applicable at the time of service.
        - Checking if the requested procedure is covered under the plan benefits.
        - Identifying and applying exclusions, co-payment clauses, and policy caps.
        - Flagging inactive, expired, or suspended policies and missing documentation.
        Your work ensures  procedural compliance with insurer rules, preventing the processing of
        ineligible or non-compliant claims.
    """),
    llm=llm
)

# 3. Medical_Necessity_Evaluator_Agent
clinical_agent = Agent(
    role="Medical Necessity Evaluation Agent",
    goal=dedent("""
        Determine whether the requested medical procedure is clinically necessary for the given diagnosis,
        based on evidence-based guidelines such as MCG and InterQual, and the sufficiency of documentation.
    """),
    backstory=dedent("""
        You are a clinical decision support specialist trained to assess medical necessity using established
        utilization review standards. Your responsibilities include:
        - Matching the submitted diagnosis (ICD code) and procedure (CPT code) against clinical guidelines
          like MCG and InterQual to assess evidence-based necessity.
        - Reviewing supporting clinical documentation (e.g., notes, lab results, imaging) to determine
          whether the procedure is justified.
        - Identifying discrepancies, overuse, or non-aligned requests where alternative conservative
          treatments are more appropriate.
        - Returning a recommendation with clear clinical rationale: Approve, Deny (with justification),
          or Request Additional Information.
        Your objective is to ensure that medical interventions are necessary, safe, and consistent
        with standard care pathways before they are approved for reimbursement.
    """),
    llm=llm
)
# 4. Cost_Impact_Assessor_Agent
cost_agent = Agent(
    role="Cost Impact and Utilization Risk Assessor",
    goal=dedent("""
        Estimate the financial burden of the requested procedure on the insurer by analyzing CPT codes,
        contract rates, and utilization thresholds, and flag procedures that are high-cost, experimental,
        or outside standard fee ranges.
    """),
    backstory=dedent("""
        You are a financial evaluation agent responsible for estimating and assessing the potential
        economic impact of medical procedures. Your work ensures the sustainability of coverage decisions
        by weighing utilization risks against policy and clinical justifications.

        Responsibilities:
        - Use CPT codes and plan parameters to generate estimated procedure costs using actuarial models
          and known fee schedules.
        - Identify procedures that exceed plan-specific cost thresholds or are considered investigational,
          non-covered, or off-label.
        - Leverage embedded cost estimation tools to provide financial output in structured form, including
          escalation flags when human financial review is required.
        - Return cost estimate, justification summary, and risk flag (e.g., high-cost, low-cost, experimental).

        Your evaluation provides critical context to support or escalate preauthorization decisions, ensuring
        financial stewardship while balancing access to care.
    """),
    tools=[cost_estimator_tool_instance],
    llm=llm
)
# 5. PreAuth_Decision_Agent
decision_agent = Agent(
    role="Final Preauthorization Decision Authority",
    goal=dedent("""
        Synthesize inputs from intake, eligibility, medical necessity, and cost analysis agents to issue
        a final, structured determination—Approve, Deny, or Request Additional Information—backed by
        transparent, multi-dimensional justification.
    """),
    backstory=dedent("""
        You are the final checkpoint in the preauthorization pipeline, responsible for issuing a decision
        that reflects the combined evaluations of upstream agents. Your primary duty is to balance clinical
        appropriateness, financial stewardship, and policy alignment to ensure high-quality care and compliance.

        Responsibilities:
        - Aggregate structured outputs from the Intake Agent, Policy Validation Agent, Medical Necessity Evaluator,
          and Cost Impact Assessor.
        - Evaluate alignment across all findings and determine whether all criteria for approval are met.
        - If approval is justified, clearly indicate the rationale referencing coverage, clinical need,
          and cost viability.
        - If denial is necessary, provide a reasoned, respectful, and guideline-backed explanation.
        - If information is insufficient, specify exactly what is missing (e.g., additional clinical notes,
          updated cost estimate, etc.).
        - Generate a structured, provider-facing JSON output that includes: `decision_status`, `decision_reason`,
          `required_documents` (if any), and `case_id`.

        Your decision finalizes the request lifecycle, supports audit traceability, and promotes fairness
        and transparency in healthcare utilization management.
    """),
    llm=llm
)

# Tasks
task1 = Task(
    description=dedent("""
        Ingest a preauthorization request provided in JSON format. Perform the following in order:

        1. Extract and standardize the following fields from the input dictionary:
           - patient_name (string)
           - age (integer)
           - diagnosis_test_results (string or list of strings)
           - ICD_code (string)
           - CPT_code (string)
           - estimated_cost (float or string)
           - insurer (string)
           - policy_number (string)

        2. Use the `JsonValidatorTool` to validate:
           - All required fields are present
           - Fields are non-empty and match expected types
           - Report any missing or malformed entries

        3. Generate a case ID using the `CreateUUIDTool` and record ingestion metadata:
           - Current UTC timestamp
           - Source format (assume it is passed via context or as an argument)

        4. Construct and return a single structured JSON object in the format shown below.
        Keep output minimal, flat, and machine-readable.

        Avoid any reasoning or explanation — only perform extraction, validation, and formatting.
    """),
    expected_output=dedent("""
                           Is a JSON with below format:
        {
            "case_id": "<UUID>",
            "ingestion_timestamp": "<UTC timestamp>",
            "source_format": "JSON",
            "completeness_validated_fields": {
                "patient_name": "...",
                "age": ...,
                "diagnosis_test_results": "...",
                "ICD_code": "...",
                "CPT_code": "...",
                "estimated_cost": "...",
                "insurer": "...",
                "policy_number": "..."
            },
            "validation_status": "success" | "error",
            "validation_issues": [ "Missing CPT_code", "Invalid age format", ... ]
        }
    """),
    agent=intake_agent
)

task2 = Task(
    description=dedent("""
        Using the structured JSON from the intake step, verify the patient’s insurance policy eligibility
        for the requested procedure. Perform the following checks:

        - Match member ID and policy number (if provided) against known formats or validation logic.
        - Determine whether the policy is currently active and within its coverage period.
        - Check if the requested CPT-coded procedure is included in the plan's covered services.
        - Apply exclusions, policy caps, and co-payment or deductible clauses where applicable.
        - Identify any red flags, such as inactive/suspended policies or missing documentation
          required for authorization (e.g., referral letters, prior conditions).

        Return a clear validation summary with eligibility decision, reasons, and any recommended next steps.
        
    """),
    expected_output=dedent("""
        A structured response in the following format:
        {
            "coverage_status": "eligible" | "ineligible" | "partially_covered" | "needs_review",
            "reasons": [
                "CPT code 99214 is excluded under this plan",
                "Policy is inactive since 2024-11-01",
                ...
            ],
            "policy_checks": {
                "member_id_valid": true,
                "policy_active": false,
                "procedure_covered": false,
                "co_pay_applicable": true,
                "exclusions_applied": ["outpatient surgery"]
            },
            "missing_documents": ["Referral letter from PCP", "Policy enrollment form"],
            "recommendation": "Escalate to human reviewer" | "Proceed to next step"
        }
    """),
    agent=policy_agent
)

task3 = Task(
    description=dedent("""
        Using the diagnosis (ICD code), procedure (CPT code), and clinical documentation provided,
        assess whether the requested procedure is medically necessary based on industry-standard
        utilization guidelines (e.g., MCG, InterQual).

        Steps:
        - Match the diagnosis and requested procedure against relevant evidence-based criteria from
          MCG or InterQual to determine alignment.
        - Evaluate whether the supporting documentation (e.g., progress notes, labs, imaging, treatment history)
          adequately justifies the need for the procedure.
        - Determine whether conservative alternatives were attempted or if the procedure is first-line.
        - Identify red flags such as elective use, overuse, or deviation from standard care pathways.

        Provide a structured clinical necessity evaluation with your final recommendation.
    """),
    expected_output=dedent("""
        {
            "necessity_status": "medically_necessary" | "not_medically_necessary" | "insufficient_documentation",
            "clinical_guideline_reference": "MCG: GI-202.4" | "InterQual: Musculoskeletal/Surgery/..." | "Other",
            "justification_summary": "Procedure aligns with standard treatment for condition X after failure of Y.",
            "supporting_documents_reviewed": ["Progress note dated 2025-06-12", "MRI report"],
            "recommendation": "Proceed to cost analysis" | "Request additional clinical information" | "Deny"
        }
    """),
    agent=clinical_agent
)

task4 = Task(
    description=dedent("""
        Conduct a financial risk assessment based on the requested procedure's CPT code and
        applicable insurance plan data.

        Steps:
        - Estimate the cost of the procedure using cost reference tools and fee schedules associated
          with the CPT code.
        - Determine if the procedure is categorized as high-cost, investigational, or experimental.
        - Evaluate whether the estimated cost exceeds the plan-specific coverage threshold or utilization cap.
        - Recommend escalation to a human financial reviewer if the risk exceeds acceptable policy limits.

        Ensure that your output is structured and provides a clear rationale behind cost risk classifications
        and escalation needs.
    """),
    expected_output=dedent("""
        {
            "estimated_cost": 1250,
            "cost_category": "standard" | "high-cost" | "experimental",
            "exceeds_threshold": true | false,
            "threshold_reference": "Plan Limit: ₹1000 per outpatient surgery",
            "recommendation": "Proceed to final decision" | "Escalate for manual review",
            "rationale": "Cost exceeds plan limit and is flagged as high-cost based on internal benchmarks."
        }
    """),
    agent=cost_agent
)

task5 = Task(
    description=dedent("""
        Synthesize structured outputs from the four upstream agents:
        - Intake Agent
        - Policy Validation Agent
        - Medical Necessity Evaluator
        - Cost Impact Assessor

        Assess alignment across eligibility, clinical appropriateness, documentation completeness, and
        financial viability. Based on this aggregation, make a final determination on the preauthorization
        request using the following decision classes:
        - "Approved"
        - "Denied"
        - "Request Additional Information"

        The decision should be supported with a clear justification referencing findings from previous steps.
        If requesting more information, specify the exact missing or incomplete elements. Your response should
        be structured in a format that can be sent directly to the provider for clarity, compliance, and audit.
    """),
    expected_output=dedent("""
        {
            "case_id": "<UUID from intake>",
            "final_decision": "Approved" | "Denied" | "Request Additional Information",
            "decision_rationale": [
                "Policy active and CPT 99213 is covered.",
                "Procedure deemed medically necessary for diagnosis E11.9 per InterQual guidelines.",
                "Estimated cost within plan threshold."
            ],
            "provider_response_message": "Your request has been approved. All clinical and policy conditions met."  # Or denial/information request message
        }
    """),
    agent=decision_agent
)

# Create the Crew
preauth_insurerside_crew = Crew(
    agents=[intake_agent],# policy_agent, clinical_agent, cost_agent, decision_agent],
    tasks=[task1],# task2, task3, task4, task5],
    process=Process.sequential,
    verbose=False)

# ---------- EXECUTION ----------
if __name__ == "__main__":
    preauth_json = {
        "preauth_request" : {
            "patient_name": "Rajesh Kumar",
            "age": 45,
            "policy_number": "STAR-HI-20394857",
            "insurer": "Star Health Insurance.",
            "diagnosis": "Osteoarthritis",
            "ICD_code": "M17.12",
            "CPT_code": "29881",
            "estimated_cost": 65000,
            "treatment_plan": "Arthroscopy, knee, surgical; meniscectomy with 3-day hospital stay",
        },
    }
    
try:
    result = preauth_insurerside_crew.kickoff(inputs=preauth_json)
    print("\n=== Final Decision Output ===")
    print(result)
except Exception as e:
    print(f"An error occurred during the crew execution: {e}")
    



=== Final Decision Output ===
<think>
Alright, let's tackle this task step by step. I need to ingest a preauthorization request in JSON format and process it through several steps: extraction, validation, ID generation, and structuring.

First, extracting fields from the input dictionary. The user provided an example JSON structure, so I'll follow that pattern. The input is something like:

{
  "patient_name": "John Doe",
  "age": 30,
  "diagnosis_test_results": [" diagnose as: tallisoid and wolverine muscle loss"],
  "ICD_code": "129", 
  "CPT_code": "85112-04-00000",
  "estimated_cost": "$5,000.00",
  "insurer": "Insurer Name",
  "policy_number": " policy_number"
}

I'll extract each field one by one, ensuring I capture all necessary details.

Next is validation using `JsonValidatorTool`. The requirements are that all required fields exist and match expected types. I need to check if the extracted data fits these criteria. For example, age must be an integer, CPT_code a string, etc.